In [0]:
#08 Audit and Lineage 
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col

spark = SparkSession.builder.getOrCreate()

# Paths for our demo tables
src = "/tmp/delta_audit_src"
dst = "/tmp/delta_audit_child"

# 1) Clean up any previous runs
dbutils.fs.rm(src, recurse=True)
dbutils.fs.rm(dst, recurse=True)

# 2) Create source table (version 0)
spark.range(0, 3) \
     .withColumn("val", lit("alpha")) \
     .write.format("delta") \
     .mode("overwrite") \
     .save(src)

# 3) Append more data (version 1)
spark.range(3, 5) \
     .withColumn("val", lit("beta")) \
     .write.format("delta") \
     .mode("append") \
     .save(src)

# 4) Create a child table via CTAS (child version 0)
spark.sql(f"CREATE TABLE delta.`{dst}` AS SELECT * FROM delta.`{src}`")

# 5) Inspect Audit History on source_tbl
print("=== SOURCE TABLE HISTORY ===")
spark.sql(f"DESCRIBE HISTORY delta.`{src}`").show(truncate=False)

# 6) Inspect Audit History (and lineage) on child_tbl
print("=== CHILD TABLE HISTORY ===")
spark.sql(f"DESCRIBE HISTORY delta.`{dst}`").show(truncate=False)